## Problem 1 - Max Flow Modeling

A Dungeon Master (DM) is trying to plan for the next three campaigns she is running. She runs campaigns for four different groups, and she is hoping to finish the groups' campaigns in the next six months.

* Group 1's campaign must be completed no later than 4 months from now and will require 20 total hours of play-time 
* Group 2's campaign must be completed no later than 5 months from now and will require 18 total hours of play-time 
*  Group 3's campaign must be completed no later than 6 months from now and will require 13 total hours of play-time 
* Group 4's campaign must be completed no later than 3 months from now and will require 14 total hours of play-time

Each month, 12 hours are available for playing games. During a given month, no more than 8 hours can be spent on the same campaign.

Formulate a _maximum flow problem_ whose solution will determine whether or not all four campaigns can be completed on time.  Your answer should clearly draw the network, the source node, the sink node, and capacity on each arc in the network.  Upload a picture of the network with your solution.


In [1]:
using JuMP, Clp

# incidence matrix 
# (rows are nodes, columns are arcs, entries represent whether arc enters (-1) or leaves (1) each node)
A = [1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0    #node src
    -1  0  0  0  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0    #node c1
     0 -1  0  0  0  0  0  0  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0    #node c2
     0  0 -1  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  0  0  0  0  0  0  0  0  0    #node c3
     0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0    #node c4
     0  0  0  0 -1  0  0  0 -1  0  0  0  0 -1  0  0  0  0  0 -1  0  0  1  0  0  0  0  0    #node m1
     0  0  0  0  0 -1  0  0  0 -1  0  0  0  0 -1  0  0  0  0  0 -1  0  0  1  0  0  0  0    #node m2
     0  0  0  0  0  0 -1  0  0  0 -1  0  0  0  0 -1  0  0  0  0  0 -1  0  0  1  0  0  0    #node m3
     0  0  0  0  0  0  0 -1  0  0  0 -1  0  0  0  0 -1  0  0  0  0  0  0  0  0  1  0  0    #node m4
     0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0  0  0  0  0  0  0  0  1  0    #node m5
     0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  1    #node m6
     0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1]   #node snk

# dummy arc
A = [A [-1;0;0;0;0;0;0;0;0;0;0;1]]

# supply and demand are all 0
b = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# costs should be 0 on every arc except dummy
# -1 on dummy arc
c = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]

# capacities on each arc. make dummy arc capacity "big enough"
cap = [20, 18, 13, 14, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 300]

m = Model(Clp.Optimizer)

# variables representing how much flow we send on each arc
@variable(m, x[1:29] >= 0)

# constraints balance flow into and out of each node
@constraint(m, A*x .== b)

# don't exceed arc capacity
@constraint(m, x .<= cap)

# minimize the total cost (same as maximizing flow through dummy arc)
@objective(m, Min, sum(c[i]*x[i] for i in 1:29))

# solve this instance of the max flow problem
optimize!(m)

# print out the flow on each arc along with total flow
println("Max flow: ", -objective_value(m)) # need to take negative
println("Flow on each arc: ")
for i in 1:28
    println("arc ", i, ": ", value(x[i]))
    end
    
println("Dummy: ", value(x[29]))

Max flow: 65.0
Flow on each arc: 
arc 1: 20.0
arc 2: 18.0
arc 3: 13.0
arc 4: 14.0
arc 5: 0.0
arc 6: 4.0
arc 7: 8.0
arc 8: 8.0
arc 9: 5.0
arc 10: 0.0
arc 11: 1.0
arc 12: 4.0
arc 13: 8.0
arc 14: 1.0
arc 15: 0.0
arc 16: 0.0
arc 17: 0.0
arc 18: 4.0
arc 19: 8.0
arc 20: 6.0
arc 21: 8.0
arc 22: 0.0
arc 23: 12.0
arc 24: 12.0
arc 25: 9.0
arc 26: 12.0
arc 27: 12.0
arc 28: 8.0
Dummy: 65.0
Coin0506I Presolve 10 (-31) rows, 27 (-2) columns and 54 (-33) elements
Clp0006I 0  Obj 0 Dual inf 3.999996 (4)
Clp0006I 21  Obj -65
Clp0000I Optimal - objective value -65
Coin0511I After Postsolve, objective -65, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -65 - 21 iterations time 0.002, Presolve 0.00


## Problem 2 - The Duality of Plushies


Assemble-an-Animal is currently producing four plushies: pandas, panthers, pangolins, and penguins.

The profit contribution, labor hours, and raw material usage (in square feet) per plushie for each type
of animal are given below:


| Plushie| Profit (\$)|Labor (Hr.)| Raw Material (ft$^2$)|
|:---|:---|:---|:---|
|Panda| 70 | 2 | 5|
|Panther |110 |3 | 5|
|Pangolin | 300 | 3 | 10|
|Penguin | 250 | 5 | 15|


A maximum of 10,000 labor hours and 35,000 square feet of raw material are available annually. Each
panda plushie spends an average of 0.25 year in storage; panther plushies an average of 1 year; pangolin plushies, an average of 2 years; penguin plushies, an average of 3.5 years.  The warehouse can handle an average inventory level of 5,000 total plushies. Determine how much of each type of plushie should be produced annually to maximize Assemble-an-Animal's profit.

A linear program model for this problem is:

$\begin{alignat*}{5}
\max \ & z = & 70x_1 &+& 110x_2 &+& 300x_3 &+&  250x_4 & \\
\text{s.t. } & & 2x_1 &+& 3x_2 &+&3x_3&+&5 x_4 & \leq 10000 \\
     & & 5x_1 &+& 5x_2 &+&10x_3 &+&15x_4 & \leq 35000 \\
	  & & 0.25 x_1 &+& x_2 &+&2x_3 &+& 3.5 x_4 & \leq 5000\\
      & & \qquad \qquad x_1 \geq 0, & &\ x_2 \geq 0, & &x_3 \geq 0, & &x_4 \geq 0
\end{alignat*}$

where $x_j$ is the number of plushies of type $j$ to produce each year.

In the sensitivity analysis questions below, answer each question _independently_ (e.g., when answering part (c), consider only the changes suggested in part (c), not those in addition to the ones considered in part (b)).

(a) Solve this model in Julia/JuMP and give the optimal primal and dual solutions, and the optimal objective function value. 


In [2]:
##optimal primal

using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, x1 >= 0)
@variable(m, x2 >= 0)
@variable(m, x3 >= 0)
@variable(m, x4 >= 0)

@objective(m, Max, 70*x1 + 110*x2 + 300*x3 + 250*x4) 

@constraint(m, chrs, 2*x1 + 3*x2 + 3*x3 + 5*x4 <= 10000)
@constraint(m, cmats, 5*x1 + 5*x2 + 10*x3 + 15*x4 <= 35000)
@constraint(m, cinv, 0.25*x1 + x2 + 2*x3 + 3.5*x4 <= 5000)


optimize!(m)   


println(m)
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))
println("x4 = ", value(x4))
println("profit =  \$", objective_value(m))

Max 70 x1 + 110 x2 + 300 x3 + 250 x4
Subject to
 chrs : 2 x1 + 3 x2 + 3 x3 + 5 x4 ≤ 10000
 cmats : 5 x1 + 5 x2 + 10 x3 + 15 x4 ≤ 35000
 cinv : 0.25 x1 + x2 + 2 x3 + 3.5 x4 ≤ 5000
 x1 ≥ 0
 x2 ≥ 0
 x3 ≥ 0
 x4 ≥ 0

x1 = 1538.4615384615388
x2 = 0.0
x3 = 2307.6923076923076
x4 = 0.0
profit =  $800000.0
Coin0506I Presolve 3 (0) rows, 4 (0) columns and 12 (0) elements
Clp0006I 0  Obj 0 Dual inf 420.70276 (4)
Clp0006I 2  Obj 800000
Clp0000I Optimal - objective value 800000
Clp0032I Optimal objective 800000 - 2 iterations time 0.002


In [3]:
##optimal dual
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, λ[1:3] >= 0)

@constraint(m, 2λ[1] + 5λ[2] + 0.25λ[3] >= 70)
@constraint(m, 3λ[1] + 5λ[2] + λ[3] >= 110)
@constraint(m, 3λ[1] + 10λ[2] + 2λ[3] >= 300)
@constraint(m, 5λ[1] + 15λ[2] + 3.5λ[3] >= 250)

# objective is to minimize the upper bound on the primal solution
@objective(m, Min, 10000*λ[1] + 35000*λ[2] + 5000*λ[3])

# solve this instance of the Top Brass dual
optimize!(m)

# print the dual model and solution
println(m)

println("dual variables: ", value.(λ))
println("Optimal objective: ", objective_value(m))

Min 10000 λ[1] + 35000 λ[2] + 5000 λ[3]
Subject to
 2 λ[1] + 5 λ[2] + 0.25 λ[3] ≥ 70
 3 λ[1] + 5 λ[2] + λ[3] ≥ 110
 3 λ[1] + 10 λ[2] + 2 λ[3] ≥ 300
 5 λ[1] + 15 λ[2] + 3.5 λ[3] ≥ 250
 λ[1] ≥ 0
 λ[2] ≥ 0
 λ[3] ≥ 0

dual variables: [20.0, 0.0, 119.99999999999999]
Optimal objective: 799999.9999999999
Coin0506I Presolve 4 (0) rows, 3 (0) columns and 12 (0) elements
Clp0006I 0  Obj 0 Primal inf 82.666666 (4)
Clp0006I 2  Obj 800000
Clp0000I Optimal - objective value 800000
Clp0032I Optimal objective 800000 - 2 iterations time 0.002


In [4]:
##double check
println(-dual(chrs))
println(-dual(cmats))
println(-dual(cinv)) 

19.999999999999996
-0.0
120.00000000000001


(b) What is the maximum amount Assemble-an-Animal should be willing to pay for an additional hour of labor?

In [5]:
println("Max value of additonal hour of labor: \$", value(λ[1]))

Max value of additonal hour of labor: $20.0


(c) The marketing department is considering running an advertising campaign that would increase the profit per panther plushie by \\$10 and panda plushie by \\$15.  Provide an estimate of the new optimal profit ($z^*_{NEW}$) after this change and indicate if your estimate is a lower or upper bound on the new optimal profit. Your answer should be of the form $z^*_{NEW} \geq \underline{\phantom{aaaa}}$ or  $z^*_{NEW} \leq \underline{\phantom{aaaa}}$, where you fill in a number in the blank.

In [6]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, x1 >= 0)
@variable(m, x2 >= 0)
@variable(m, x3 >= 0)
@variable(m, x4 >= 0)

@objective(m, Max, (70 + 15)*x1 + (110 + 10)*x2 + 300*x3 + 250*x4) 

@constraint(m, chrs, 2*x1 + 3*x2 + 3*x3 + 5*x4 <= 10000)
@constraint(m, cmats, 5*x1 + 5*x2 + 10*x3 + 15*x4 <= 35000)
@constraint(m, cinv, 0.25*x1 + x2 + 2*x3 + 3.5*x4 <= 5000)


optimize!(m)   

println(m)
println("profit z{NEW} <=  \$", objective_value(m))
println(-dual(chrs))
println(-dual(cmats))
println(-dual(cinv)) 

Max 85 x1 + 120 x2 + 300 x3 + 250 x4
Subject to
 chrs : 2 x1 + 3 x2 + 3 x3 + 5 x4 ≤ 10000
 cmats : 5 x1 + 5 x2 + 10 x3 + 15 x4 ≤ 35000
 cinv : 0.25 x1 + x2 + 2 x3 + 3.5 x4 ≤ 5000
 x1 ≥ 0
 x2 ≥ 0
 x3 ≥ 0
 x4 ≥ 0

profit z{NEW} <=  $823076.923076923
29.230769230769226
-0.0
106.15384615384616
Coin0506I Presolve 3 (0) rows, 4 (0) columns and 12 (0) elements
Clp0006I 0  Obj 0 Dual inf 453.77398 (4)
Clp0006I 2  Obj 823076.92
Clp0000I Optimal - objective value 823076.92
Clp0032I Optimal objective 823076.9231 - 2 iterations time 0.002


Compared to the optimal dual solutions found in (a), the maximum amount Assemble-an-Animal is willing to pay for labor per hour increases, while the amount in inventory decreases and best of all, profit increases. This would be an upper bound, however, as the dual values are changing as a result of manipulating the objective function, which assumes that constraints remain the same to achieve the added profits per respective plushie. Therefore, $z^*_{NEW} \leq $823076.923076923.

(d) Assemble-an-Animal is considering a facility redesign that would decrease the labor availability
by 1000 hours, but increase the raw material availability by 4000 square feet. Provide an estimate of the new optimal profit ($z^*_{NEW}$) after this change and indicate if your estimate is a lower or upper bound on the new optimal profit. Your answer should be of the form $z^*_{NEW} \geq \underline{\phantom{aaaa}}$ or  $z^*_{NEW} \leq \underline{\phantom{aaaa}}$, where you fill in a number in the blank.

In [7]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, x1 >= 0)
@variable(m, x2 >= 0)
@variable(m, x3 >= 0)
@variable(m, x4 >= 0)

@objective(m, Max, 70*x1 + 110*x2 + 300*x3 + 250*x4) 

@constraint(m, chrs, 2*x1 + 3*x2 + 3*x3 + 5*x4 <= (10000 - 1000))
@constraint(m, cmats, 5*x1 + 5*x2 + 10*x3 + 15*x4 <= (35000 + 4000))
@constraint(m, cinv, 0.25*x1 + x2 + 2*x3 + 3.5*x4 <= 5000)


optimize!(m)   


println(m)
println("profit z{NEW} >=  \$", objective_value(m))
println(-dual(chrs))
println(-dual(cmats))
println(-dual(cinv)) 

Max 70 x1 + 110 x2 + 300 x3 + 250 x4
Subject to
 chrs : 2 x1 + 3 x2 + 3 x3 + 5 x4 ≤ 9000
 cmats : 5 x1 + 5 x2 + 10 x3 + 15 x4 ≤ 39000
 cinv : 0.25 x1 + x2 + 2 x3 + 3.5 x4 ≤ 5000
 x1 ≥ 0
 x2 ≥ 0
 x3 ≥ 0
 x4 ≥ 0

profit z{NEW} >=  $779999.9999999999
19.999999999999996
-0.0
120.00000000000001
Coin0506I Presolve 3 (0) rows, 4 (0) columns and 12 (0) elements
Clp0006I 0  Obj 0 Dual inf 420.70276 (4)
Clp0006I 2  Obj 780000
Clp0000I Optimal - objective value 780000
Clp0032I Optimal objective 780000 - 2 iterations time 0.002


Similar to the rationale of (c), we can compare the duals of the new dual values and those of the default function and constraints. It is apparent that the duals for labor hours, materials, and inventory are unchanged, but with a significantly smaller optimal objective value, which is indicative of it being a lower bound of the new optimal profit. Thus, $z^*_{NEW} \geq $779999.9999999999.

## Problem 3 - Duality and complementarity 


Consider the following linear program:

\begin{align}
  \min z = 2 x_1 + x_3 & \label{eq:lp-duality-6}  \\
  \mbox{s.t.} \quad x_1 + x_2 - x_3 &\geq 5\nonumber\\ 
  x_1 - 2 x_2 + 4 x_3 &\geq 8\nonumber\\ 
  x_1, x_2, x_3 &\geq 0 \nonumber
\end{align}

(1) Give the dual of this linear program.


Dual:

\begin{align}
  \max z_d = 5λ_1 + 8λ_2 & \label{eq:lp-duality-6}  \\
  \mbox{s.t.} \quad λ_1 + λ_2 &\leq 2\nonumber\\ 
  λ_1 -2λ_2 &\leq 0\nonumber\\ 
  -λ_1 + 4λ_2 &\leq 1\nonumber\\
  λ_1, λ_2 &\geq 0 \nonumber
\end{align}

In [8]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, λ[1:2] >= 0)

@constraint(m, λ[1] + λ[2] <= 2)
@constraint(m, λ[1] - 2*λ[2] <= 0)
@constraint(m, -λ[1] + 4*λ[2] <= 1)

# objective is to minimize the upper bound on the primal solution
@objective(m, Max, 5*λ[1] + 8*λ[2])

# solve this instance of the Top Brass dual
optimize!(m)

# print the dual model and solution
println(m)

println("dual variables: ", value.(λ))
println("Dual objective: ", objective_value(m))

Max 5 λ[1] + 8 λ[2]
Subject to
 λ[1] + λ[2] ≤ 2
 λ[1] - 2 λ[2] ≤ 0
 -λ[1] + 4 λ[2] ≤ 1
 λ[1] ≥ 0
 λ[2] ≥ 0

dual variables: [1.0, 0.5]
Dual objective: 9.0
Coin0506I Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Dual inf 13 (2)
Clp0006I 3  Obj 9
Clp0000I Optimal - objective value 9
Clp0032I Optimal objective 9 - 3 iterations time 0.002


(2) Find a complementary dual solution to the primal solution:
$ \hat{x} := (\hat{x}_1, \hat{x}_2, \hat{x}_3) = \left(\frac{28}{5}, 0, \frac{3}{5} \right) $


$ \hat{x}_1 + \hat{x}_2 - \hat{x}_3 = 28/5 + 0 - 3/5 = 25/5 = 5 ≥ 5 $



$ \hat{x}_1 - 2\hat{x}_2 + 4\hat{x}_3 = 28/5 - 2(0) + 4(3/5) = 28/5 + 12/5 = 40/5 = 8 ≥ 8 $

because both constraints hold with equality, dual constraints also hold by equality and can be transformed: 

λ1 + λ2 = 2
λ1 - 2λ2 = 0    => λ1 = 2λ2
-λ1 + 4λ2 = 1

Substitute: 

-2λ2 + 4λ2 = 1   =>  2λ2 = 1   => λ2 = 1/2

λ1 = 2(1/2)   => λ1 = 1


complementary dual solution to primal solution $ \hat{x} := (\hat{x}_1, \hat{x}_2, \hat{x}_3) = \left(\frac{28}{5}, 0, \frac{3}{5} \right) $ is λ1 = 1, λ2 = 1/2

(3) Is your dual solution feasible?  What can you conclude about $\hat{x}$?  

As proven above:

$ \hat{x}_1 + \hat{x}_2 - \hat{x}_3 = 28/5 + 0 - 3/5 = 25/5 = 5 ≥ 5 $



$ \hat{x}_1 - 2\hat{x}_2 + 4\hat{x}_3 = 28/5 - 2(0) + 4(3/5) = 28/5 + 12/5 = 40/5 = 8 ≥ 8 $

Therefore, it is feasible, and $\hat{x}$ is optimal, as it produces the same dual values as the solver


(4)   Draw the feasible region to the dual you derived in (1).  Find the optimal dual solution by the graphical method.  


(5) Use complementary slackness and your dual solution from (4) to fimd the optimal primal solution.

dual solutions from the graph were λ1 = 1, λ2 = 1/2 

plug them into the dual objective function: 

5λ1 + 8λ2 = 5(1) + 8(1/2) = 9

This answer is also supported by the prior linear program that was run. 